In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import display, Markdown, Latex
import itertools


def lorentzian(f, I, f_0, S, a, b):
    return I/(1+((f-f_0)/S)**2)+a*f+b     #Lorentziana con correzione lineare

def frequenza(T,A,B,C,D,E,TC):
    return A*(1+(B*C/(D*(1-(T/TC)**2)**(1/2)))*(1/(np.tanh((E/C)*(1-(T/TC)**2)**(1/2)))))**(-1/2)

prova_1 = pd.read_csv("Car_Prova_1.txt")   #file txt da caricare manualmente
prova_2 = pd.read_csv("Car_Prova_2.txt")
prova_3 = pd.read_csv("Car_Prova_3.txt")
prova_4 = pd.read_csv("Car_Prova_4.txt")
prova_5 = pd.read_csv("Car_prova_5.txt")
prova_6 = pd.read_csv("Salto_Def.txt")

guess_1 = [0.8, 1, -0.08, 0.01, 0.15]  #guess da inserire manualmente
guess_2 = [0.8, 1, -0.08, 0.01, 0.15]
guess_3 = [0.8, 1, -0.08, 0.01, 0.15]
guess_4 = [0.8, 1, -0.08, 0.01, 0.15]
guess_5 = [0.8, 1, -0.08, 0.01, 0.15]
guess_6 = [126.34,1.91,1.63,21816,0.037,90.1] 

temp = prova_6["Temperatura potenziometro"].values
freq = prova_6["Frequenze"].values
x = [prova_6["Temperatura potenziometro"][i] for i in range(7, len(freq))]
z = [prova_6["Frequenze"][i] for i in range(7, len(temp))]
guesses = [guess_1, guess_2, guess_3, guess_4]
prove = [prova_1, prova_2, prova_3, prova_4]

freqs, guads = [x["Frequenze normalizzate"].values for x in prove], [x['Guadagni normalizzati'].values for x in prove]

p_1, cov_1 = curve_fit(lorentzian, freqs[0], guads[0], guesses[0])   #fit di ogni coppia di valori frequenza/guadagno
p_2, cov_2 = curve_fit(lorentzian, freqs[1], guads[1], guesses[1])
p_3, cov_3 = curve_fit(lorentzian, freqs[2], guads[2], guesses[2])
p_4, cov_4 = curve_fit(lorentzian, freqs[3], guads[3], guesses[3])
p_6, cov_6 = curve_fit(frequenza, z, x, guess_6)

ps, covs = [p_1, p_2, p_3, p_4], [cov_1, cov_2, cov_3, cov_4]     #set di parametri del fit

f_ris_1=p_1[1]*115.0       #frequenze di risonanza
f_ris_2=p_2[1]*115.0
f_ris_3=p_3[1]*124.0
f_ris_4=p_4[1]*94.0

y_1, y_2, y_3, y_4 = [], [], [], []        #liste inizializzate vuote che saranno riempite con i valori di fit dell'asse y

Set_Color_1 = ["r", "m", "c", "coral"]
Set_Color_2 = ["black", "g", "b", "gray"] 

ys = [y_1, y_2, y_3, y_4]

f_risonanze = [f_ris_1, f_ris_2, f_ris_3, f_ris_4]

for m,k,h in zip(f_risonanze, ps, range(1,len(ps)+1)):
    tuplex=(h,)+tuple(k)
    display(Latex('Fitting parameters test %5.0f: $\quad$ I = %5.3f, $\quad$ $\omega_0$ = %5.3f $\quad$ S = %5.3f, $\quad$ a = %5.3f, $\quad$ b = %5.3f' %tuplex))
    display(Latex('La frequenza di risonanza è: %5.2f Hz' %m))
    print("\n")       #fitting parameters test

for k,l,m,f in zip(ys, prove, ps, freqs):
    for j in range(len(f)):
        k.append(lorentzian(l["Frequenze normalizzate"][j], m[0], m[1], m[2], m[3], m[4])) #riempie le liste vuote y tramite fit con lorentziana e parametri


y_6=[]

for j in range(7,len(freq)):
        y_6.append(frequenza(prova_5["Temperatura potenziometro"][j], p_6[0], p_6[1], p_6[2], p_6[3], p_6[4],p_6[5]))        
             
points, fit = "g","b"

plt.plot(prova_6["Temperatura potenziometro"], prova_6["Frequenze"], "bo", color = points, label = 'Exp. data')
plt.plot(x, y_6, fit, label = 'theoretical curve fit')
plt.xlabel('$T[°K]$')
plt.ylabel('$f_r\,[kHz]$')
plt.legend()

print("\n")

fig, axes = plt.subplots(2, 2, squeeze = False)    #crea array 2D per rappresentare i grafici
plt.subplots_adjust(left=0, bottom=None, right=3, top=3, wspace=0.2, hspace=0.15)  #regola la posizione dei grafici
  
indici = list(itertools.product(range(axes.shape[0]),range(axes.shape[1]))) #crea gli indici dell'array di figure

for k,y,m,h,t,a in zip(prove, ys, ps, indici, Set_Color_1, Set_Color_2):
    axes[h[0]][h[1]].plot(k['Frequenze normalizzate'], k['Guadagni normalizzati'], "o", color = a, label = 'Exp. data')
    axes[h[0]][h[1]].plot(k['Frequenze normalizzate'], y, t, label = '\nFit parameters:\n\n I = %5.3f\n $\omega_0$ = %5.3f\n S = %5.3f\n a = %5.3f\n b = %5.3f' % tuple(m))
    axes[h[0]][h[1]].set(xlabel = '$\\omega/\omega_0$', ylabel = 'Normalized Gain')
    axes[h[0]][h[1]].legend(loc="upper right")


KeyError: 'Frequenze normalizzate'